
L'obiettivo del progetto è apprendere mediante una rete neurale la trasformazione da punti del piano espressi in coordinate polari ad una rappresentazione basata su di una griglia discreta di dimensione 10x10, dove la cella della griglia ha valore 1 se contiene il punto, e 0 altrimenti.

Il dataset supervisionato è fornito in questo notebook nella forma di una generatore. Il generatore deve essere considerato come una "scatola nera" il cui comportamento deve essere appreso. 

Dovete progettare una rete neurale in grado di raggiungere una accuratezza del 95%. Questa è una condizione necessaria per superare l'esame, ma l'accuratezza non influisce in altro modo sulla valutazione.  

I modelli che raggiungono l'accuratezza attesa saranno invece valutati in modo inversamente proporzionale al numero dei loro parametri: **più il modello è piccolo, meglio è.**


**Attenzione**: Qualunque soluzione che tragga vantaggio, diretto o indiretto, da meta-conoscenza relativa al generatore sarà automaticamente bocciato.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model

Veniamo al generatore. Questo restituisce delle triple della forma
((theta,rho),out) dove (theta,rho) sono le coordinate polari di un punto nel primo quadrante del piano, e out è una mappa 10x10 con "1" in correspondenza alla cella che contiene il punto, e "0" altrimenti.

Settando  flat=True, la mappa 10x10 viene appiattita ad un vettore di dimensione 100. Potete utilizzare questa variante, se preferite. Nessuna altra modifica del generatore è ammessa. 

In [2]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Creiamo una istanza del generatore con una griglia di dimensione 3x4

In [3]:
g1,g2 = 3,4
gen = polar_generator(1,grid=(g1,g2),noise=0.0)

... e osserviamo qualche esempio

In [4]:
(theta,rho),maps = next(gen)
for i,map in enumerate(maps):
  #let us compute the cartesian coordinates
  x = np.cos(theta[i])*rho[i]
  y = np.sin(theta[i])*rho[i]
  print("x coordinate (row): {}".format(int(x*g1)))
  print("y coordinate (col): {}".format(int(y*g2)))
  print("map:")
  print(np.reshape(map,(g1,g2)))

x coordinate (row): 0
y coordinate (col): 1
map:
[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Utile esercizio: aggiungete rumore al generatore e verificate l'effetto sulla
"ground truth".

# Cosa consegnare

Ai fini del progetto dovete lavorare con la **griglia di default di dimensione 10x10, e con il rumore di default .002**

il generatore deve essere trattato come una scatola nera: non modificatelo e non sfruttate la sua semantica, che si suppone ignota. Potete lavorare in modlaità "flat", se preferite.

Dovete:

1.   definire una funzione per il calcolo della accuratezza (potete prendere ispirazione dal cocice della cella precedente) 
2.   definire una rete neurale che prende in input theta e rho e restituisce out
3.  misurare l'accuratezza della rete, che deve essere maggiore o uguale del 95%; l'accuratezza deve essere misurata su almeno 20000 dati
4. perfezionare il modello cercando di diminuire il più possibile il numero dei parametri mantenendo una accuratezza superiore al 95%. Solo la vostra rete migliore deve essere consegnata.

Dovete consegnare un UNICO notebook eseguibile su colab, che contenga il codice della rete, il suo sommario con il numero dei parametri, la storia di training, il codice per il calcolo della accuratezza e la sua valutazione sulla vostra rete.

**N.B.** L'accuratezza deve essere superiore o uguale a 95%, ma non influisce in altro modo sulla valutazione. Il vostro punteggio dipenderà unicamente dal numero dei parametri: più è piccolo e più la vostra vaalutazione sarà elevata.  

#Buon lavoro!





In [35]:
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras import activations
from keras import losses
from sklearn.model_selection import train_test_split
import keras.backend as K

In [54]:
def accuracy(true_maps: tf.Tensor, my_maps: tf.Tensor) -> float:
  return tf.math.count_nonzero(tf.equal(true_maps, my_maps)) / \
    tf.cast(tf.size(true_maps), tf.int64)

In [46]:
n_train = 6000
n_test = 2000
batch_size = 128

g1,g2 = 10,10
gen = polar_generator(n_train+n_test,grid=(g1,g2),noise=0.02,flat=True)
(theta,rho),y = next(gen)
x=np.array([i for i in zip(theta,rho)])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=n_test/(n_train+n_test), shuffle=True, random_state=1)

In [61]:
def loss_fn(y_true, y_pred):
  return losses.categorical_crossentropy(y_true, y_pred)

In [62]:
model = keras.Sequential([
    layers.Flatten(),
    layers.Dense(4, activation=activations.relu),
    layers.Dense(8, activation=activations.swish),
    layers.Dense(16, activation=activations.swish),
    #tf.keras.layers.Dropout(0.2),
    layers.Dense(100, activation=activations.softmax)
])
model.build((None, 2))
model.summary()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-1),
    loss=loss_fn,
    metrics=[accuracy]
)

history = model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=batch_size
)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_18 (Flatten)        (None, 2)                 0         
                                                                 
 dense_70 (Dense)            (None, 4)                 12        
                                                                 
 dense_71 (Dense)            (None, 8)                 40        
                                                                 
 dense_72 (Dense)            (None, 16)                144       
                                                                 
 dense_73 (Dense)            (None, 100)               1700      
                                                                 
Total params: 1,896
Trainable params: 1,896
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
47/47 [==============================] 

In [63]:
print('Accuracy: {:.2f}%'.format(accuracy(model.predict(x_test), y_test) * 100))

63/63 [==============================] - 0s 1ms/step
Accuracy: 3.22%
